In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from art.estimators.classification import PyTorchClassifier
import pandas as pd
import numpy as np
import art
import torch.nn as nn
import torch.optim as optim
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, DeepFool, BoundaryAttack

# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
])

# 加载CIFAR-10数据集
trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)

# 合并训练集和测试集
combined_data = trainset.data
combined_labels = trainset.targets
combined_data = np.concatenate((combined_data, testset.data), axis=0)
combined_labels = np.concatenate((combined_labels, testset.targets), axis=0)

# 获取类别名称
class_names = trainset.classes

# 创建一个字典来存储每个类别的DataFrame
dataframes = {}

# 按标签分组数据
for label in range(len(class_names)):
    # 获取当前标签的索引
    indices = np.where(combined_labels == label)[0]
    
    # 提取对应的数据和标签
    label_data = combined_data[indices]
    label_labels = combined_labels[indices]
    
    # 创建DataFrame
    df = pd.DataFrame({
        'image': [img for img in label_data],
        'label': label_labels
    })
    
    # 用类别名称命名DataFrame
    dataframes[class_names[label]] = df

# 返回10个DataFrame



# 如果需要返回这些DataFrame，可以返回字典
# return dataframes

In [ ]:
import torchvision.models as models
resnet_name=r"D:\python\resnetcifar\model\resmodel.pth"
# 定义与保存时一致的模型结构
class ModifiedResNet18(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=False)  # 不使用预训练权重，因为我们加载的是已经训练好的模型
        self.resnet18.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet18.maxpool = torch.nn.Identity()  # 移除最大池化层

        # 获取ResNet18的特征提取部分
        self.features = torch.nn.Sequential(*list(self.resnet18.children())[:-1])

        # 添加MLP和全连接层
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5)
        )
        self.fc = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.mlp(x)
        x = self.fc(x)
        return x

# 初始化模型
model = ModifiedResNet18()

# 加载保存的模型权重
model_path = resnet_name # 替换为你的模型文件路径
model.load_state_dict(torch.load(model_path))


In [ ]:
def create_art_classifier(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
    )
    return classifier
classifier=create_art_classifier(model)

In [ ]:
# 删除airplane这一类
if 'airplane' in dataframes:
    del dataframes['airplane']
    print("Deleted 'airplane' class from dataframes.")
else:
    print("'airplane' class not found in dataframes.")
if 'automobile' in dataframes:
    del dataframes['automobile']
    print("Deleted automobile class from dataframes.")  
if 'bird' in dataframes:
    del dataframes['bird']
    print("Deleted bird class from dataframes.")  
if 'cat' in dataframes:
    del dataframes['cat']
    print("Deleted cat class from dataframes.")  
if 'deer' in dataframes:
    del dataframes['deer']
    print("Deleted deer class from dataframes.")  
if 'dog' in dataframes:
    del dataframes['dog']
    print("Deleted dog class from dataframes.") 
if 'frog' in dataframes:
    del dataframes['frog']
    print("Deleted frog class from dataframes.") 

In [5]:
import os

# 创建保存目录（如果不存在）
output_dir = r"D:\python\resnetcifar\adv\deepfool"  # 指定保存目录
os.makedirs(output_dir, exist_ok=True)

#ART 使用的DeepFool攻击
max_iter=10
epsilon=8/255
def attack_dataframe_deepfool(df, max_iter=50):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建DeepFool攻击对象
    attack = DeepFool(
        classifier=classifier,
        max_iter=max_iter,
        epsilon=epsilon,
        batch_size=128
    )

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy())

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 100 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df

# 对整个字典中的所有类别进行DeepFool攻击
def attack_all_dataframes_deepfool(dataframes, max_iter=50):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_deepfool(df, max_iter)
        attacked_dataframes[class_name] = attacked_df
        
        # 创建一个新的DataFrame来存储攻击后的数据
        df_to_save = attacked_df.copy()
        df_to_save['class_name'] = class_name  # 添加类别名称列

        # 将图像数据转换为字符串格式以便保存
        df_to_save['image'] = df_to_save['image'].apply(lambda x: x.tobytes())

        # 保存到CSV文件
        filename = f"{class_name}_deepfool.csv"
        save_path = os.path.join(output_dir, filename)  # 指定保存路径
        df_to_save.to_csv(save_path, index=False)
        print(f"Data for class '{class_name}' saved to {save_path}")
    
    return attacked_dataframes

# 执行DeepFool攻击
dfs_deepfool = attack_all_dataframes_deepfool(dataframes, max_iter=50)

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

...

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

Processed 6000 images
Data for class 'truck' saved to D:\python\resnetcifar\adv\deepfool\truck_deepfool.csv


In [ ]:
# 使用ART的BIM攻击
def attack_dataframe_bim(df, epsilon=8/255, max_iter=10, eps_step=2/255):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建BIM攻击对象
    attack = BasicIterativeMethod(
        estimator=classifier, 
        eps=epsilon,
        eps_step=eps_step,
        max_iter=max_iter
    )

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy())

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 1000 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df

# 对整个字典中的所有类别进行BIM攻击
def attack_all_dataframes_bim(dataframes, epsilon=8/255, max_iter=10, eps_step=2/255):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_bim(df, epsilon, max_iter, eps_step)
        attacked_dataframes[class_name] = attacked_df
    return attacked_dataframes

bims = attack_all_dataframes_bim(dataframes, epsilon=8/255, max_iter=10, eps_step=2/255)
save_attacked_dataframes_to_csv(bims,"bims.csv")

In [ ]:
# 使用ART的DeepFool攻击
def attack_dataframe_deepfool(df, max_iter=10, epsilon=0.1):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建DeepFool攻击对象
    attack = DeepFool(
        classifier=classifier,
        max_iter=max_iter,
        epsilon=epsilon,
        batch_size=1
    )

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy())

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 1000 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df

# 对整个字典中的所有类别进行DeepFool攻击
def attack_all_dataframes_deepfool(dataframes, max_iter=10, epsilon=0.1):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_deepfool(df, max_iter, epsilon)
        attacked_dataframes[class_name] = attacked_df
    return attacked_dataframes

deepfools = attack_all_dataframes_deepfool(dataframes, max_iter=10, epsilon=0.1)
save_attacked_dataframes_to_csv(deepfools,"deepfools.csv")

In [ ]:
# 使用ART的Boundary Attack（黑盒攻击）
def attack_dataframe_boundary(df, targeted=False, max_iter=100, delta=0.01):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建Boundary攻击对象
    attack = BoundaryAttack(
        classifier=classifier,
        targeted=targeted,
        max_iter=max_iter,
        delta=delta
    )

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy(), y=np.array([label]))

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 1000 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df

# 对整个字典中的所有类别进行Boundary Attack
def attack_all_dataframes_boundary(dataframes, targeted=False, max_iter=100, delta=0.01):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_boundary(df, targeted, max_iter, delta)
        attacked_dataframes[class_name] = attacked_df
    return attacked_dataframes

boundaries = attack_all_dataframes_boundary(dataframes, targeted=False, max_iter=100, delta=0.01)
save_attacked_dataframes_to_csv(boundaries,"boundarys.csv")

In [ ]:
# 使用ART的FGSM攻击
def attack_dataframe_art(df, epsilon=8/255):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建FGSM攻击对象
    attack = FastGradientMethod(estimator=classifier, eps=epsilon)

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy())

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 1000 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df

In [ ]:
# 对整个字典中的所有类别进行FGSM攻击
def attack_all_dataframes(dataframes, epsilon=8/255):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_art(df, epsilon)
        attacked_dataframes[class_name] = attacked_df
    return attacked_dataframes

In [ ]:
fgsms=attack_all_dataframes(dataframes, epsilon=8/255)

In [ ]:
#存储和恢复
import pandas as pd
import numpy as np

# 将attacked_dataframes保存到CSV文件
def save_attacked_dataframes_to_csv(attacked_dataframes, filename='fgsm.csv'):
    # 创建一个空的DataFrame来存储所有类别的数据
    all_data = pd.DataFrame(columns=['image', 'label', 'class_name'])

    # 遍历每个类别的DataFrame并添加类别名称列
    for class_name, df in attacked_dataframes.items():
        df['class_name'] = class_name  # 添加类别名称列
        all_data = pd.concat([all_data, df], ignore_index=True)

    # 将图像数据转换为字符串格式以便保存
    all_data['image'] = all_data['image'].apply(lambda x: x.tobytes())

    # 保存到CSV文件
    all_data.to_csv(filename, index=False)
    print(f"Data saved to {filename}")


save_attacked_dataframes_to_csv(fgsms)

In [ ]:
import pandas as pd
import numpy as np

# 从CSV文件中读取数据并恢复attacked_dataframes
def load_attacked_dataframes_from_csv(filename='fgsm.csv'):
    # 读取CSV文件
    all_data = pd.read_csv(filename)

    # 将图像数据从字符串格式转换回NumPy数组
    all_data['image'] = all_data['image'].apply(lambda x: np.frombuffer(eval(x), dtype=np.uint8).reshape(32, 32, 3))

    # 按类别名称分组并创建字典
    attacked_dataframes = {}
    for class_name in all_data['class_name'].unique():
        class_data = all_data[all_data['class_name'] == class_name]
        df = class_data[['image', 'label']].reset_index(drop=True)
        attacked_dataframes[class_name] = df

    return attacked_dataframes

# 示例用法
if __name__ == "__main__":
    # 从CSV文件中读取数据
    attacked_dataframes = load_attacked_dataframes_from_csv('fgsm.csv')

    # 打印恢复后的字典
    for class_name, attacked_df in attacked_dataframes.items():
        print(f"Recovered DataFrame for class '{class_name}':")
        print(attacked_df.head())
        print("\n")

In [ ]:
from art.attacks.evasion import CarliniL2Method
# 使用ART的CW攻击
def attack_dataframe_cw(df, confidence=0.0, max_iter=500):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建CW攻击对象
    attack = CarliniL2Method(classifier=classifier, confidence=confidence, max_iter=max_iter)

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy())

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 100 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df


In [ ]:
# 对整个字典中的所有类别进行CW攻击
def attack_all_dataframes_cw(dataframes, confidence=0.0, max_iter=1000):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_cw(df[:200], confidence, max_iter)
        attacked_dataframes[class_name] = attacked_df
         # 创建一个新的DataFrame来存储攻击后的数据
        df_to_save = attacked_df.copy()
        df_to_save['class_name'] = class_name  # 添加类别名称列

        # 将图像数据转换为字符串格式以便保存
        df_to_save['image'] = df_to_save['image'].apply(lambda x: x.tobytes())

        # 保存到CSV文件
        filename = f"{class_name}_cw1.csv"
        df_to_save.to_csv(filename, index=False)
        print(f"Data for class '{class_name}' saved to {filename}")
    
    return attacked_dataframes


In [ ]:
# 删除airplane这一类
if 'airplane' in dataframes:
    del dataframes['airplane']
    print("Deleted 'airplane' class from dataframes.")
else:
    print("'airplane' class not found in dataframes.")
if 'automobile' in dataframes:
    del dataframes['automobile']
    print("Deleted automobile class from dataframes.")  
"""if 'bird' in dataframes:
    del dataframes['bird']
    print("Deleted bird class from dataframes.")  
if 'cat' in dataframes:
    del dataframes['cat']
    print("Deleted cat class from dataframes.")  
if 'deer' in dataframes:
    del dataframes['deer']
    print("Deleted deer class from dataframes.")  
if 'dog' in dataframes:
    del dataframes['dog']
    print("Deleted dog class from dataframes.")  """

In [ ]:
cws=attack_all_dataframes_cw(dataframes,0.0, max_iter=500)

In [ ]:
cws=attack_all_dataframes_cw(dataframes,0.0, max_iter=500)

In [ ]:
#PGD
from art.attacks.evasion import ProjectedGradientDescent
# 使用ART的PGD攻击
def attack_dataframe_pgd(df, epsilon=8/255, alpha=2/255, iterations=10):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建PGD攻击对象
    attack = ProjectedGradientDescent(
        estimator=classifier,
        eps=epsilon,
        eps_step=alpha,
        max_iter=iterations,
        targeted=False,
        num_random_init=1,
        batch_size=32
    )

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy())

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 1000 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df

In [ ]:
# 删除airplane这一类
if 'airplane' in dataframes:
    del dataframes['airplane']
    print("Deleted 'airplane' class from dataframes.")
else:
    print("'airplane' class not found in dataframes.")
if 'automobile' in dataframes:
    del dataframes['automobile']
    print("Deleted automobile class from dataframes.")  
if 'bird' in dataframes:
    del dataframes['bird']
    print("Deleted bird class from dataframes.")  
if 'cat' in dataframes:
    del dataframes['cat']
    print("Deleted cat class from dataframes.")  
if 'deer' in dataframes:
    del dataframes['deer']
    print("Deleted deer class from dataframes.")  
if 'dog' in dataframes:
    del dataframes['dog']
    print("Deleted dog class from dataframes.")  

In [ ]:
"""# 对整个字典中的所有类别进行PGD攻击
def attack_all_dataframes_pgd(dataframes, epsilon=8/255, alpha=2/255, iterations=10):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_pgd(df, epsilon, alpha, iterations)
        attacked_dataframes[class_name] = attacked_df
    return attacked_dataframes"""
# 对整个字典中的所有类别进行PGD攻击，并保存到CSV文件
def attack_all_dataframes_pgd(dataframes, epsilon=8/255, alpha=2/255, iterations=10):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_pgd(df, epsilon, alpha, iterations)
        attacked_dataframes[class_name] = attacked_df

        # 创建一个新的DataFrame来存储攻击后的数据
        df_to_save = attacked_df.copy()
        df_to_save['class_name'] = class_name  # 添加类别名称列

        # 将图像数据转换为字符串格式以便保存
        df_to_save['image'] = df_to_save['image'].apply(lambda x: x.tobytes())

        # 保存到CSV文件
        filename = f"{class_name}_pgd.csv"
        df_to_save.to_csv(filename, index=False)
        print(f"Data for class '{class_name}' saved to {filename}")
    
    return attacked_dataframes

In [ ]:
pgds = attack_all_dataframes_pgd(dataframes, epsilon=8/255, alpha=2/255, iterations=10)

In [6]:
#合并所有pgd
import os
import glob
import pandas as pd
import numpy as np
# 将10个单独的PGD攻击后的CSV文件整合成一个文件
def merge_pgd_csv_files(output_filename='merged_pgd.csv'):
    # 获取当前目录下所有以_pgd.csv结尾的文件
    csv_files = glob.glob(r'D:\python\resnetcifar\adv\deepfool\*_deepfool.csv')
    
    # 创建一个空的DataFrame来存储所有数据
    merged_data = pd.DataFrame(columns=['image', 'label', 'class_name'])
    
    # 遍历每个CSV文件并合并数据
    for file in csv_files:
        # 读取CSV文件
        df = pd.read_csv(file)
        
        # 将图像数据从字符串格式转换回NumPy数组
        df['image'] = df['image'].apply(lambda x: np.frombuffer(eval(x), dtype=np.uint8).reshape(32, 32, 3))
        
        # 合并数据
        merged_data = pd.concat([merged_data, df], ignore_index=True)
    
    # 将图像数据转换为字符串格式以便保存
    merged_data['image'] = merged_data['image'].apply(lambda x: x.tobytes())
    
    # 保存到CSV文件
    merged_data.to_csv(output_filename, index=False)
    #print(f"Merged data saved to {output_filename}")
    
    return merged_data


merge_pgd_csv_files(output_filename=r'D:\python\resnetcifar\adv\deepfool\deepfools.csv')

,image,label,class_name
0,b'\xca\xca\xcc\xcd\xcf\xd0\xd1\xd4\xd4\xd7\xd8...,0,airplane
1,b'}z~~\x81\x81\x84\x84\x81\x83\x86\x82\x82\x85...,0,airplane
2,b'\xfa\xf7\xf7\xf8\xf9\xf8\xf8\xf8\xf9\xf9\xf8...,0,airplane
3,b'\xff\xff\xff\xb6\xb5\xff\xff\xff\xff\xfe\xff...,0,airplane
4,"b',/3554566:A:9B:<=@BI>?GEHIGIGKHJ3/2/46:<<;@B...",0,airplane
...,...,...,...
59995,b'baaabbbbbbba``^]]\\[[ZYXVUTRPNLIHeeegfffffee...,9,truck
59996,b'\xf4\xf1\xf2\xf3\xf4\xf4\xf4\xf3\xf2\xf2\xf2...,9,truck
59997,b'\xe1\xe4\xe4\xe8\xe8\xe9\xeb\xeb\xee\xe9\xe2...,9,truck
59998,b'7\x1f&Hy\xbb\x9c\x97\xc3\xd0\xe1\xf5\xf0\xb9...,9,truck


In [ ]:
#合并所有pgd
import os
import glob
import pandas as pd
import numpy as np
# 将10个单独的PGD攻击后的CSV文件整合成一个文件
def merge_cw_csv_files(output_filename='cw1s.csv'):
    # 获取当前目录下所有以_pgd.csv结尾的文件
    csv_files = glob.glob(r'D:\python\resnetcifar\adv\cw\*_cw1.csv')
    
    # 创建一个空的DataFrame来存储所有数据
    merged_data = pd.DataFrame(columns=['image', 'label', 'class_name'])
    
    # 遍历每个CSV文件并合并数据
    for file in csv_files:
        # 读取CSV文件
        df = pd.read_csv(file)
        
        # 将图像数据从字符串格式转换回NumPy数组
        df['image'] = df['image'].apply(lambda x: np.frombuffer(eval(x), dtype=np.uint8).reshape(32, 32, 3))
        
        # 合并数据
        merged_data = pd.concat([merged_data, df], ignore_index=True)
    
    # 将图像数据转换为字符串格式以便保存
    merged_data['image'] = merged_data['image'].apply(lambda x: x.tobytes())
    
    # 保存到CSV文件
    merged_data.to_csv(output_filename, index=False)
    #print(f"Merged data saved to {output_filename}")
    
    return merged_data


merge_cw_csv_files(output_filename="cw1s.csv")

In [ ]:
# 从合并后的CSV文件中读取数据并恢复attacked_dataframes
def load_attacked_dataframes_from_csv(filename='merged_pgd.csv'):
    # 读取CSV文件
    all_data = pd.read_csv(filename)
    
    # 将图像数据从字符串格式转换回NumPy数组
    all_data['image'] = all_data['image'].apply(lambda x: np.frombuffer(eval(x), dtype=np.uint8).reshape(32, 32, 3))
    
    # 按类别名称分组并创建字典
    attacked_dataframes = {}
    for class_name in all_data['class_name'].unique():
        class_data = all_data[all_data['class_name'] == class_name]
        df = class_data[['image', 'label']].reset_index(drop=True)
        attacked_dataframes[class_name] = df
    
    return attacked_dataframes
pgds = load_attacked_dataframes_from_csv("pgds.csv")

In [ ]:
#AutoAttack攻击
from art.attacks.evasion import AutoAttack
def attack_dataframe_autoattack(df, epsilon=8/255):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建AutoAttack攻击对象
    attack = AutoAttack(
        estimator=classifier,
        norm=np.inf,  # 使用np.inf表示L∞范数
        eps=epsilon,
        eps_step=epsilon / 4,
        targeted=False,
        batch_size=512
    )

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy())

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 1000 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df

In [ ]:
# 对整个字典中的所有类别进行AutoAttack攻击
def attack_all_dataframes_autoattack(dataframes, epsilon=8/255):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_autoattack(df, epsilon)
        attacked_dataframes[class_name] = attacked_df
        
                # 创建一个新的DataFrame来存储攻击后的数据
        df_to_save = attacked_df.copy()
        df_to_save['class_name'] = class_name  # 添加类别名称列

        # 将图像数据转换为字符串格式以便保存
        df_to_save['image'] = df_to_save['image'].apply(lambda x: x.tobytes())

        # 保存到CSV文件
        filename = f"{class_name}_auto.csv"
        df_to_save.to_csv(filename, index=False)
        print(f"Data for class '{class_name}' saved to {filename}")
    return attacked_dataframes

In [ ]:
autos=attack_all_dataframes_autoattack(dataframes, epsilon=8/255)